In [52]:
import pandas as pd
df = pd.read_csv(r"C:\Users\hp\OneDrive\codes\python\Learn_ML\1. Datasets\cars.csv")
df.head()

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


In [53]:
import numpy as np

np.random.seed(42)
missing_km_indices = np.random.choice(df.index, size=int(0.05*len(df)), replace=False)
df.loc[missing_km_indices, 'km_driven'] = np.nan

# Introduce missing values in 'owner' column (1% missing values)
missing_owner_indices = np.random.choice(df.index, size=int(0.01*len(df)), replace=False)
df.loc[missing_owner_indices, 'owner'] = np.nan

In [54]:
df.isnull().sum()

brand              0
km_driven        406
fuel               0
owner             81
selling_price      0
dtype: int64

```python
Column Transformers
1. Missing value handling
2. Encoding categorical columns
3. Scaling
4. Feature Selection
5. Model Building
6. Prediction
```

In [103]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest,chi2

In [56]:
X_train,X_test,y_train,y_test = train_test_split(df.iloc[:,0:4],df.iloc[:,-1],test_size=0.2,random_state=42)

In [71]:
# imputation transformer
impute = ColumnTransformer([
    ("imputing_km_driven",SimpleImputer(strategy="median"),[1]),
    ("imputing_owner",SimpleImputer(strategy="most_frequent"),[3])
],remainder="passthrough")

In [72]:
# encoding transfomer
encode = ColumnTransformer(
    [
        ("onehot",OneHotEncoder(sparse_output=False,handle_unknown="ignore"),[0,2]),
        ("ordinal",OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),[3])
    ],remainder="passthrough"
)

In [73]:
# Scaling transformer
scaler = ColumnTransformer(
    [
        ("scaling_features",MinMaxScaler(),slice(0,38))
    ]
)
# the no of columns will increase to 38 after the previous two transfomration operations

In [74]:
#feature selection
selector = SelectKBest(score_func=chi2, k=10)

In [104]:
rf = RandomForestClassifier()

In [105]:
pipe = Pipeline([
    ("imputer", impute),
    ("encoder", encode),
    ("scaler", scaler),
    ("selector", selector),
    ("random forest", rf)
])

In [106]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputing_km_driven',
                                                  SimpleImputer(strategy='median'),
                                                  [1]),
                                                 ('imputing_owner',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [3])])),
                ('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0, 2]),
                                                 ('ordinal',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  [3])])),
                ('scaler',
                 ColumnTransformer(transformers=[('scaling_features',
                                                  MinMaxScaler(),
                                                  slice(0, 38, None))])),
                ('selector',
                 SelectKBest(score_func=<function chi2 at 0x0000021F5469FBA0>)),
                ('random forest', RandomForestClassifier())])

In [107]:
pipe.get_feature_names_out

<bound method Pipeline.get_feature_names_out of Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputing_km_driven',
                                                  SimpleImputer(strategy='median'),
                                                  [1]),
                                                 ('imputing_owner',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [3])])),
                ('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [0, 2]),
                                                 ('o

In [108]:
pipe.named_steps

{'imputer': ColumnTransformer(remainder='passthrough',
                   transformers=[('imputing_km_driven',
                                  SimpleImputer(strategy='median'), [1]),
                                 ('imputing_owner',
                                  SimpleImputer(strategy='most_frequent'),
                                  [3])]),
 'encoder': ColumnTransformer(remainder='passthrough',
                   transformers=[('onehot',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [0, 2]),
                                 ('ordinal',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  [3])]),
 'scaler': ColumnTransformer(transformers=[('scaling_features', MinMaxScaler(),
                                  slice(0

In [118]:
input_data = pd.DataFrame(
    [['Maruti', 145500.0, 'Diesel', 'First Owner']],
    columns=df.columns[0:4]
)

# Use the pipeline to predict
prediction = pipe.predict(input_data)
print(prediction)

[300000]


In [116]:
from sklearn.model_selection import GridSearchCV

# Update the params dictionary to include the step name 'random forest'
params = {
	'random forest__max_depth': [1, 2, 3, 4, 5, None]
}

grid = GridSearchCV(pipe, params, cv=5, scoring='neg_mean_squared_error')
grid.fit(X_train, y_train)

c:\Users\hp\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('imputing_km_driven',
                                                                         SimpleImputer(strategy='median'),
                                                                         [1]),
                                                                        ('imputing_owner',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [3])])),
                                       ('encoder',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore...
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         [3])])),
                                       ('scaler',
                                        ColumnTransformer(transformers=[('scaling_features',
                                                                         MinMaxScaler(),
                                                                         slice(0, 38, None))])),
                                       ('selector',
                                        SelectKBest(score_func=<function chi2 at 0x0000021F5469FBA0>)),
                                       ('random forest',
                                        RandomForestClassifier())]),
             param_grid={'random forest__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='neg_mean_squared_error')

In [117]:
grid.best_score_

-728527419607.2467